# Carnatic Beat Detection: what is that beat?


### Overview

This project develops an ML/AI classifier that identifies the Carnatic beat cycle (taalam) of a mridangam solo



### Goal

Given a clip of a drum solo, identify the taalam (beat cycle) in which it is performed.

In this notebook, we will do an exploratory data analysis with only 2 taalam classes: aadi talam (8-beat cycle) and misra-chapu talam (7-beat cycle), to investigate if classic ML/AI techniques can successfully classify these two types with the data at hand.

### Data

In [51]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns
import os
import random
import math

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, TargetEncoder, LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer, TransformedTargetRegressor, ColumnTransformer
from sklearn.feature_selection import SequentialFeatureSelector, RFE
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression 
from sklearn.metrics import mean_squared_error, accuracy_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier

import time

In [52]:
# Load the data
beatsDf1 = pd.read_csv("data/beats-3Class.csv")
beatsDf1.sample(7)

,FileName,Beat,1,2,3,4,5,6,7,8,...,491,492,493,494,495,496,497,498,499,500
41,MisraChapu-1007,M,0.003768,0.007246,0.014203,0.018841,0.023188,0.027246,0.030435,0.042029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,MisraChapu-1012,M,0.000290,0.002323,0.004355,0.006388,0.009001,0.013066,0.015970,0.019454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Palghat raghu misra chapu - 103,M,0.001448,0.004922,0.011581,0.014765,0.018240,0.025188,0.028662,0.031847,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,MisraChapu105,M,0.000579,0.004053,0.008686,0.012160,0.015055,0.019977,0.024320,0.026346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,MisraChapu-2,M,0.004353,0.008416,0.010737,0.013059,0.017702,0.021764,0.023506,0.026698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,MisraChapu-1,M,0.002025,0.006655,0.012153,0.014178,0.018229,0.021991,0.033275,0.038484,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,KhandaChapu2027,K,0.003196,0.005811,0.008425,0.011331,0.014817,0.018884,0.022371,0.028472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
beatsDf1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Columns: 502 entries, FileName to 500
dtypes: float64(500), object(2)
memory usage: 455.1+ KB


#### Cleanup 1:
We will set the number of numeric (time-lapse) features to be 250. Drop features with names > 250.

In [54]:
# We will drop features with names 251 -- 500
featuresToDrop = []
for nn in range(251, 501):
    featuresToDrop.append(str(nn))

In [55]:
# Drop the columns above
beatsDf1 = beatsDf1.drop(featuresToDrop, axis = 1)

In [56]:
beatsDf1.sample(7)

,FileName,Beat,1,2,3,4,5,6,7,8,...,241,242,243,244,245,246,247,248,249,250
47,MisraChapu-1013,M,0.009059,0.016949,0.023670,0.031268,0.038574,0.045880,0.049971,0.053185,...,0.789889,0.793103,0.797779,0.800994,0.802747,0.805961,0.808884,0.812098,0.814144,0.815605
87,KhandaChapu2017,K,0.000000,0.003205,0.006702,0.010781,0.013112,0.017191,0.021270,0.024476,...,0.735140,0.738054,0.741550,0.745047,0.747378,0.750291,0.752914,0.754662,0.756702,0.759033
99,KhandaChapu2029,K,0.002023,0.005202,0.013584,0.017630,0.019942,0.025723,0.028035,0.033815,...,0.741908,0.744798,0.746821,0.750000,0.751734,0.753468,0.756936,0.759827,0.762717,0.765318
17,Aadi-1017,A,0.001445,0.004624,0.008671,0.013584,0.017052,0.019942,0.022543,0.025145,...,0.793064,0.796821,0.798555,0.800289,0.803757,0.806069,0.808671,0.811272,0.815318,0.817630
3,Aadi-1003,A,0.003196,0.006392,0.009587,0.028181,0.031668,0.035445,0.038640,0.049680,...,0.938698,0.941894,0.944509,0.946833,0.948286,0.949739,0.951772,0.957873,0.961650,0.964265
77,MisraChapu-1025,M,0.002033,0.003775,0.006098,0.008420,0.012485,0.015970,0.022358,0.028165,...,0.939315,0.943961,0.948606,0.951510,0.953833,0.957317,0.959640,0.961963,0.963705,0.966609
79,KhandaChapu2009,K,0.000000,0.002328,0.004948,0.007276,0.010477,0.013679,0.016589,0.018917,...,0.750873,0.753492,0.755821,0.758149,0.761350,0.763970,0.767753,0.770955,0.773283,0.775029


In [57]:
beatsDf1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Columns: 252 entries, FileName to 250
dtypes: float64(250), object(2)
memory usage: 228.5+ KB


#### Cleanup 2:
Drop rows with NaNs. These are really short clips and probably will not contain sufficient data for analysis. There is no meaningful way to fill in missing values.

In [58]:
beatsDf_full = beatsDf1.dropna()

In [59]:
beatsDf_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 0 to 115
Columns: 252 entries, FileName to 250
dtypes: float64(250), object(2)
memory usage: 199.6+ KB


In [60]:
beatsDf_full.sample(5)

,FileName,Beat,1,2,3,4,5,6,7,8,...,241,242,243,244,245,246,247,248,249,250
46,MisraChapu-1012,M,0.000290,0.002323,0.004355,0.006388,0.009001,0.013066,0.015970,0.019454,...,0.868467,0.874855,0.881243,0.883566,0.887050,0.889373,0.892857,0.895470,0.898084,0.900407
27,Aadi-1027,A,0.000581,0.003486,0.005230,0.006682,0.009006,0.013074,0.016560,0.020046,...,0.833818,0.839338,0.842243,0.843986,0.845729,0.848925,0.851540,0.855607,0.859965,0.865776
99,KhandaChapu2029,K,0.002023,0.005202,0.013584,0.017630,0.019942,0.025723,0.028035,0.033815,...,0.741908,0.744798,0.746821,0.750000,0.751734,0.753468,0.756936,0.759827,0.762717,0.765318
111,MisraChapu106,M,0.000869,0.004635,0.008691,0.011008,0.012746,0.015643,0.019988,0.022885,...,0.813731,0.815469,0.820684,0.824739,0.838644,0.843569,0.845597,0.848783,0.852549,0.854867
45,MisraChapu-1011,M,0.003187,0.006373,0.009560,0.011877,0.016512,0.018830,0.020568,0.029258,...,0.754345,0.756952,0.760429,0.763036,0.765933,0.769119,0.772016,0.773754,0.775492,0.776941


In [61]:
# Drop the "FileName" column
beatsDf_full = beatsDf_full.drop("FileName", axis = 1)

In [62]:
beatsDf_full.value_counts('Beat')

Beat
A    36
M    34
K    31
Name: count, dtype: int64

In [63]:
# For the current state of the data, we have only one 'K' beat. 
#Drop it so that we have a 2-fold (binary) classification situation.
beatsDf_2Class = beatsDf_full.drop(beatsDf_full[beatsDf_full.Beat == 'K'].index)

In [64]:
beatsDf_2Class.value_counts('Beat')

Beat
A    36
M    34
Name: count, dtype: int64

### Split data into training and test sets

In [65]:
# Data: indepndent and target variables
X = beatsDf_2Class.drop(['Beat'], axis = 1)

# target
labelEnc = LabelEncoder()
y = labelEnc.fit_transform(beatsDf_2Class['Beat'])

In [66]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [67]:
# Preprocessing pipeline for numerical features
numeric_feats = []
for nn in range(1, 251):
    numeric_feats.append(str(nn))    

In [68]:
preprocPipe = ColumnTransformer(
    transformers=[
        ('numeric', StandardScaler(), numeric_feats)
    ])

In [69]:
rand_state = 44
# Data, split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = rand_state)

### Building a model for each classifier

In [70]:
warnings.filterwarnings('ignore')

# Model pipelines
logRegPipe = make_pipeline(preprocPipe, LogisticRegression(max_iter=10000, random_state = rand_state))
decTreePipe = make_pipeline(preprocPipe, DecisionTreeClassifier(random_state = rand_state))
knnPipe = make_pipeline(preprocPipe, KNeighborsClassifier())
svmPipe = make_pipeline(preprocPipe, SVC(random_state = rand_state))
logRegCVPipe = make_pipeline(preprocPipe, LogisticRegressionCV(cv=5, random_state = rand_state, max_iter=10000))

bgClassifierPipe = make_pipeline(preprocPipe, BaggingClassifier(DecisionTreeClassifier(random_state = rand_state), oob_score = True))
rfClassifierPipe = make_pipeline(preprocPipe, RandomForestClassifier(random_state=rand_state))
adaPipe = make_pipeline(preprocPipe, AdaBoostClassifier(DecisionTreeClassifier(), random_state = rand_state))

pipelines = [logRegPipe, decTreePipe, knnPipe, svmPipe, logRegCVPipe, bgClassifierPipe, rfClassifierPipe, adaPipe]

### Model evaluation

In [71]:
# Evaluating the models
model_performance = []

for pipe in pipelines:
        #Start a timer
        start_time = time.time()
        
        # fit the data
        pipe.fit(X_train, y_train)
        
        #End the timer, get elapsed time
        end_time = time.time()
        fit_time = end_time - start_time

        # Make a prediction, measure the accuracy
        y_pred = pipe.predict(X_test)
        score = accuracy_score(y_test, y_pred)
        
        modelName = type(pipe._final_estimator).__name__

        model_performance.append({
            'Model': modelName,
            'Score': score,
            'Time': fit_time
            })
        

In [72]:

# Dataframe out of the results
performDf = pd.DataFrame(model_performance)

In [73]:
performDf

,Model,Score,Time
0,LogisticRegression,0.500000,0.036930
1,DecisionTreeClassifier,0.428571,0.019945
2,KNeighborsClassifier,0.571429,0.005982
3,SVC,0.571429,0.005984
4,LogisticRegressionCV,0.500000,0.526559
5,BaggingClassifier,0.500000,0.090793
6,RandomForestClassifier,0.714286,0.161531
7,AdaBoostClassifier,0.428571,0.021971


In [80]:
pFig1 = px.bar(performDf, x = 'Model', y = 'Score', color = 'Model')
pFig1.update_layout(
            title={
            'text' : 'Model Accuracy - 2 Classes',
            'x':0.4,
            'xanchor': 'center'
        })
pFig1.show()

### Evaluation

#### Model Accuracy
Many models fared only at the level of random chance. Howver, RandomForestClassifier was able to scre about 70%. This is without any hyperparameter tuning.

#### Caveat
The above analysis was done with a very small dataset! The results might change as more data is generated and brought into the analysis.